# DES script for simulating hospital operations

In [493]:
import simpy
import networkx as nx
import numpy as np
import random
import pickle
import json

## Load the graph

In [494]:
with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/edited_graph.pkl', 'rb') as f:
    G = pickle.load(f)

## Load the shortest patient paths

In [495]:
with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/new_all_real_medical_path_1.pkl', 'rb') as f:
    all_real_medical_path_1 = pickle.load(f)

with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/new_all_real_medical_path_2.pkl', 'rb') as f:
    all_real_medical_path_2 = pickle.load(f)

with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/new_all_real_medical_path_3.pkl', 'rb') as f:
    all_real_medical_path_3 = pickle.load(f)

with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/new_all_real_pediatric_path_1.pkl', 'rb') as f:
    all_real_pediatric_path_1 = pickle.load(f)

with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/new_all_real_pediatric_path_2.pkl', 'rb') as f:
    all_real_pediatric_path_2 = pickle.load(f)

with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/new_all_real_pediatric_path_3.pkl', 'rb') as f:
    all_real_pediatric_path_3 = pickle.load(f)

In [496]:
print(all_real_medical_path_1)
print(all_real_medical_path_2)
print(all_real_medical_path_3)
print(all_real_pediatric_path_1)
print(all_real_pediatric_path_2)
print(all_real_pediatric_path_3)

[[44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 309, 71, 309, 11, 104, 301, 74, 302, 1, 0, 49, 266, 89, 297, 10], [44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 306, 85, 306, 11, 104, 301, 74, 302, 1, 0, 49, 266, 89, 297, 10], [44, 333, 7, 335, 0, 1, 152, 1, 324, 119, 471, 21, 328, 86, 328, 21, 471, 119, 324, 1, 0, 49, 266, 89, 297, 10], [44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 301, 74, 302, 1, 0, 49, 266, 89, 297, 10], [44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 308, 138, 308, 11, 104, 301, 74, 302, 1, 0, 49, 266, 89, 297, 10], [44, 333, 7, 335, 0, 1, 152, 1, 324, 119, 471, 21, 330, 139, 330, 21, 471, 119, 324, 1, 0, 49, 266, 89, 297, 10], [44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 312, 170, 312, 11, 104, 301, 74, 302, 1, 0, 49, 266, 89, 297, 10], [44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 313, 207, 313, 11, 104, 301, 74, 302, 1, 0, 49, 266, 89, 297, 10], [44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 307, 208, 307, 

In [497]:
print(all_real_medical_path_1[0])
print(all_real_medical_path_1[1])
print(all_real_medical_path_1[2])
print(len(all_real_medical_path_1))
print(len(all_real_medical_path_2))
print(len(all_real_medical_path_3))
print(len(all_real_pediatric_path_1))
print(len(all_real_pediatric_path_2))
print(len(all_real_pediatric_path_3))

[44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 309, 71, 309, 11, 104, 301, 74, 302, 1, 0, 49, 266, 89, 297, 10]
[44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 306, 85, 306, 11, 104, 301, 74, 302, 1, 0, 49, 266, 89, 297, 10]
[44, 333, 7, 335, 0, 1, 152, 1, 324, 119, 471, 21, 328, 86, 328, 21, 471, 119, 324, 1, 0, 49, 266, 89, 297, 10]
24
24
24
12
12
12


# DES starts

### Initialize SimPy environment

In [498]:
env = simpy.Environment()

### Calculate each node's capacity (how many patients can it serve at one time)

#### - waiting area and corridors' capacities are 1m² per person, all other nodes' capacites are 1.

In [499]:

def get_capacity(G):

    # Extract nodes with 'name' attribute containing 'CORRIDOR' or 'WAITING'
    extracted_nodes = [node for node in G.nodes(data=True) if 'CORRIDOR' in node[1]['name'] or 'WAITING' in node[1]['name']]

    # Add 'capacity' attribute to each extracted node
    for node in extracted_nodes:
        node_index, node_data = node
        node_data['capacity'] = int(float(node_data['area']))  # Convert to float first, then to int

    # Create the capacities dictionary
    capacities = {}

    for node in G.nodes(data=True):
        node_index, node_data = node
        # If 'capacity' attribute exists, use it, otherwise assign 1
        capacities[node_index] = node_data.get('capacity', 1)

    return capacities

capacities = get_capacity(G)
print(capacities)

{0: 127, 1: 178, 2: 1, 3: 50, 4: 1, 5: 1, 6: 25, 7: 27, 8: 1, 9: 40, 10: 1, 11: 29, 12: 1, 13: 1, 14: 41, 15: 24, 16: 20, 17: 1, 18: 39, 19: 10, 20: 1, 21: 29, 22: 27, 23: 1, 24: 31, 25: 16, 26: 19, 27: 28, 28: 15, 29: 1, 30: 13, 31: 12, 32: 1, 33: 13, 34: 1, 35: 14, 36: 17, 37: 1, 38: 10, 39: 1, 40: 1, 41: 1, 42: 1, 43: 1, 44: 1, 45: 1, 46: 1, 47: 11, 48: 1, 49: 19, 50: 1, 51: 13, 52: 1, 53: 7, 54: 20, 55: 1, 56: 1, 57: 1, 58: 1, 59: 1, 60: 1, 61: 1, 62: 1, 63: 8, 64: 1, 65: 1, 66: 1, 67: 1, 68: 1, 69: 8, 70: 1, 71: 1, 72: 1, 73: 1, 74: 5, 75: 5, 76: 5, 77: 1, 78: 1, 79: 1, 80: 1, 81: 9, 82: 1, 83: 1, 84: 1, 85: 1, 86: 1, 87: 1, 88: 1, 89: 1, 90: 7, 91: 1, 92: 1, 93: 1, 94: 1, 95: 1, 96: 1, 97: 1, 98: 1, 99: 12, 100: 1, 101: 1, 102: 1, 103: 1, 104: 1, 105: 1, 106: 1, 107: 1, 108: 1, 109: 1, 110: 1, 111: 1, 112: 1, 113: 1, 114: 1, 115: 1, 116: 1, 117: 1, 118: 1, 119: 11, 120: 1, 121: 1, 122: 1, 123: 1, 124: 1, 125: 1, 126: 1, 127: 1, 128: 1, 129: 11, 130: 7, 131: 1, 132: 1, 133: 1, 134

In [500]:
# according to capacity, Create resources at nodes
resources = {node: simpy.Resource(env, capacity=capacities[node]) for node in G.nodes}

### Data tracking

In [501]:
waiting_times = {node: [] for node in G.nodes}
walking_distances = []  # Track walking distance for each patient
patient_waiting_times = []  # Track total waiting time for each patient
crowding = {node: [] for node in G.nodes} #crowdedness
patients_in_node = {node: 0 for node in G.nodes}
patient_travel_paths = {}  # Dictionary to store each patient's journey

# Track occupancy at each node over time
occupancy = {node: [] for node in G.nodes}

### Function to calculate distance between two nodes

In [502]:
def calculate_distance(node1, node2):
    pos1 = np.array(G.nodes[node1]['coords'])
    pos2 = np.array(G.nodes[node2]['coords'])
    return np.linalg.norm(pos1 - pos2)

### Function for creating Patient process (patient goes through patient path)

In [503]:
def patient(env, patient_id, path):
    total_walking_distance = 0
    total_waiting_time = 0
    journey = []  

    for i, node in enumerate(path):
        journey.append(node)  

        # Calculate walking distance 
        if i > 0:
            dist = calculate_distance(path[i-1], node)
            total_walking_distance += dist

        # calculate waiting time
        if capacities[node] > 0:  
            arrival_time = env.now
            with resources[node].request() as req:
                patients_in_node[node] += 1  # Update crowding
                yield req
                
                waiting_time = env.now - arrival_time
                total_waiting_time += waiting_time
                waiting_times[node].append(waiting_time)

        # simulate treatment (random service time: 15)
                yield env.timeout(np.random.exponential(15))  

        # calculate crowdedness (people density)
                occupancy[node].append(patients_in_node[node])

        crowding[node].append(patients_in_node[node] / float(G.nodes[node]['area']) if G.nodes[node]['area'] != 'NA' else 0)
        patients_in_node[node] -= 1  # Patient leaves the node

    # Record the walking distance, waiting time, and travel path for this patient
    walking_distances.append(total_walking_distance)
    patient_waiting_times.append(total_waiting_time)
    patient_travel_paths[f'patient_{patient_id}'] = journey  


### Function for choosing any available interaction room

Since there are multiple interaction rooms for patient to choose, 

this function ensures that patient can choose any available room, so it saves waiting time.

In [504]:
# new code for checking which nodes are multiple choice nodes
def get_path_node_info(all_path,G):

    path_info = {}
    for path_id, path in enumerate(all_path):
        
        node_info_dict = {}

        for node in path:
            node_info = G.nodes[node]
            node_info_dict[node] = node_info

        path_info['path_'+str(path_id)] = node_info_dict

    #print(len(path_info))
    return path_info

m1_path_info = get_path_node_info(all_real_medical_path_1,G)
m2_path_info = get_path_node_info(all_real_medical_path_2,G)
m3_path_info = get_path_node_info(all_real_medical_path_3,G)
p1_path_info = get_path_node_info(all_real_pediatric_path_1,G)
p2_path_info = get_path_node_info(all_real_pediatric_path_2,G)
p3_path_info = get_path_node_info(all_real_pediatric_path_3,G)

In [505]:
def get_triage_and_interaction_station(path_node_info):

    final_dict = {}
    collected_path_and_triage_station = {}
    for path_id, path_info in path_node_info.items():
        
        filtered_dict = {k: {'name': v['name']} for k, v in path_info.items() if 'WTS' in v['name'] or 'INTERACTION' in v['name']}
        final_dict[path_id] = filtered_dict
    
    for path_name, stations in final_dict.items():
        node_ids = list(stations.keys())
        collected_node_ids = [node_ids[0], node_ids[-1]]
        collected_path_and_triage_station[path_name]= collected_node_ids

    return collected_path_and_triage_station

medical_p1_triage_interaction = get_triage_and_interaction_station(m1_path_info)
medical_p2_triage_interaction = get_triage_and_interaction_station(m2_path_info)
medical_p3_triage_interaction = get_triage_and_interaction_station(m3_path_info)
pediatric_p1_triage_interaction = get_triage_and_interaction_station(p1_path_info)
pediatric_p2_triage_interaction = get_triage_and_interaction_station(p2_path_info)
pediatric_p3_triage_interaction = get_triage_and_interaction_station(p3_path_info)

In [506]:
def get_available_paths(resources,all_paths, triage_and_station):
    available_paths = []
    for path, values in triage_and_station.items():
        if resources[(values[0])].count < resources[(values[0])].capacity and resources[(values[1])].count < resources[(values[1])].capacity:  
            available_paths.append(all_paths[(int(path.split('_')[1]))])

    return available_paths


### Simulation input, e.g., path choices and the possibility of taking each path, this can also be turned into requiring path data input from users.

In [507]:

path_choices = ['medical_1', 'medical_2', 'medical_3', 
                        'pediatric_1', 'pediatric_2', 'pediatric_3']
path_weights = [0.49, 0.05, 0.01, 0.39, 0.05, 0.01]

all_paths = [all_real_medical_path_1,all_real_medical_path_2,all_real_medical_path_3,
             all_real_pediatric_path_1,all_real_pediatric_path_2,all_real_pediatric_path_3]
all_tria_and_interaction = [medical_p1_triage_interaction,medical_p2_triage_interaction,medical_p3_triage_interaction,
                            pediatric_p1_triage_interaction,pediatric_p2_triage_interaction,pediatric_p3_triage_interaction]



### Function for generating patients

In [508]:
def generate_patients(env, num_patients,all_paths,all_tria_and_interaction,path_choices,path_weights):
    for i in range(num_patients):
        # patient choose a path based on probabilities
        path_type_choice = random.choices(path_choices, weights=path_weights)[0]

        # Find the index of the chosen path
        path_index = path_choices.index(path_type_choice)
        
        # Get available paths for the chosen path
        available_paths = get_available_paths(resources, all_paths[path_index], all_tria_and_interaction[path_index])
        
        if available_paths:
            chosen_path = random.choice(available_paths)
            env.process(patient(env, i, chosen_path))
        else:
            # If no paths available, the patient waits for some time before retrying
            yield env.timeout(1)
        
        # Simulate random patient arrival time 
       

        yield env.timeout(np.random.exponential(10))  # Random patient arrival time

### Run the simulation & collect patient travel paths

In [509]:
env.process(generate_patients(env, 10,all_paths,all_tria_and_interaction,path_choices,path_weights))  # Simulate the hospital with 20 patients
env.run(until=720)  # Run the simulation for 1440 minutes

# print all patient travel paths
def extract_patient_number(key):
    return int(key.split('_')[1])

for key in sorted(patient_travel_paths.keys(), key=extract_patient_number):
    print(f"{key}, {patient_travel_paths[key]}")

patient_0, [44, 333, 7, 335, 0, 1, 324, 119, 471, 21, 470, 107, 470, 21, 471, 119, 324, 1, 302, 74, 301, 104, 11, 306, 85, 306, 11, 104, 301, 74, 302, 1, 0, 49, 266, 89, 297, 10]
patient_1, [44, 333, 7, 335, 0, 1, 324, 119, 471, 21, 470, 107, 470, 21, 471, 119, 471, 21, 472, 211, 472, 21, 471, 119, 324, 1, 0, 49, 266, 89, 297, 10]
patient_2, [44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 307, 208, 307, 11, 104, 301, 74, 302, 1, 0, 49, 266, 89, 297, 10]
patient_7, [44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 309, 71, 309, 11, 104, 301, 74, 302, 1, 0, 49, 266, 89, 297, 10]


# Assessing Difficulty in wayfinding 
### - by calculating extra walking distance

### load the random patient paths

In [510]:
with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/new_all_random_medical_path_1.pkl', 'rb') as f:
    all_real_medical_random_path_1 = pickle.load(f)

with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/new_all_random_medical_path_2.pkl', 'rb') as f:
    all_real_medical_random_path_2 = pickle.load(f)

with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/new_all_random_medical_path_3.pkl', 'rb') as f:
    all_real_medical_random_path_3 = pickle.load(f)

with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/new_all_random_pediatric_path_1.pkl', 'rb') as f:
    all_real_pediatric_random_path_1 = pickle.load(f)

with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/new_all_random_pediatric_path_2.pkl', 'rb') as f:
    all_real_pediatric_random_path_2 = pickle.load(f)

with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/new_all_random_pediatric_path_3.pkl', 'rb') as f:
    all_real_pediatric_random_path_3 = pickle.load(f)

### Calculate distance of patient path

In [511]:
# Function for calulating total distance of the path

import math

def calculate_path_distance(path, G):
    total_distance = 0.0

    # Iterate through consecutive nodes in the path
    for i in range(len(path) - 1):
        node1 = path[i]
        node2 = path[i + 1]
        
        # Get the 3D coordinates of both nodes
        x1, y1, z1 = G.nodes[node1]['coords']
        x2, y2, z2 = G.nodes[node2]['coords']
        
        # Calculate the Euclidean distance between node1 and node2
        distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2 + (z2 - z1) ** 2)
        
        # Add the distance to the total distance
        total_distance += distance

    return total_distance


In [512]:
# Function for calulating average distance of multiple paths

def get_total_distance_allPath(all_path,G):
    distances = []
    for path in all_path:
        single_distance = calculate_path_distance(path,G)
        distances.append(single_distance)
    final_total_distance = sum(distances)
    average = final_total_distance / len(distances)

    return average

In [513]:
# Average distance of the random paths
all_distance_m1 = get_total_distance_allPath(all_real_medical_random_path_1,G)
print(all_distance_m1)

all_distance_m2 = get_total_distance_allPath(all_real_medical_random_path_2,G)
print(all_distance_m2)

all_distance_m3 = get_total_distance_allPath(all_real_medical_random_path_3,G)
print(all_distance_m3)

all_distance_p1 = get_total_distance_allPath(all_real_pediatric_random_path_1,G)
print(all_distance_p1)  

all_distance_p2 = get_total_distance_allPath(all_real_pediatric_random_path_2,G)
print(all_distance_p2) 

all_distance_p3 = get_total_distance_allPath(all_real_pediatric_random_path_1,G)
print(all_distance_p3) 

24431.384940265634
23822.884196410654
28308.635505665923
21568.505246506502
38831.68177714813
21568.505246506502


In [514]:
# average distance of shortest paths

all_shortest_distance_m1 = get_total_distance_allPath(all_real_medical_path_1,G)
print(all_shortest_distance_m1)

all_shortest_distance_m2 = get_total_distance_allPath(all_real_medical_path_2,G)
print(all_shortest_distance_m2)

all_shortest_distance_m3 = get_total_distance_allPath(all_real_medical_path_3,G)
print(all_shortest_distance_m3)

all_shortest_distance_p1 = get_total_distance_allPath(all_real_pediatric_path_1,G)
print(all_shortest_distance_p1)

all_shortest_distance_p2 = get_total_distance_allPath(all_real_pediatric_path_2,G)
print(all_shortest_distance_p2)

all_shortest_distance_p3 = get_total_distance_allPath(all_real_pediatric_path_3,G)
print(all_shortest_distance_p3)

157.51595594963723
179.45764113973368
192.9740593324609
172.62135514002588
194.56304033012233
208.07945852284956


### Extra walking distance

In [515]:
def get_extra_walking_distance(random_d1,random_d2,random_d3,
                               random_p1,random_p2,random_p3,
                               shortest_d1,shortest_d2,shortest_d3,
                               shortest_p1,shortest_p2,shortest_p3):
    random_average_distance = 0.49*random_d1+0.05*random_d2+0.01*random_d3+0.39*random_p1+0.05*random_p2+0.01*random_p3
    shortest_average_distance = 0.49*shortest_d1+0.05*shortest_d2+0.01*shortest_d3+0.39*shortest_p1+0.05*shortest_p2+0.01*shortest_p3

    extra_distance = random_average_distance - shortest_average_distance

    return extra_distance

In [516]:
extra_distance = get_extra_walking_distance(all_distance_m1,all_distance_m2,all_distance_m3,
                                            all_distance_p1,all_distance_p2,all_distance_p3,
                                            all_shortest_distance_m1,all_shortest_distance_m2,all_shortest_distance_m3,
                                            all_shortest_distance_p1,all_shortest_distance_p2,all_shortest_distance_p3)

print(extra_distance)

23847.378656895384


# Evaluation mechanism of simulation results

## Aggregating simulation results

In [517]:
# After simulation, analyze the collected data
average_walking_distance = np.mean(walking_distances)
average_patient_waiting_time = np.mean(patient_waiting_times)

# people density at each node
Average_crowding_at_each_node = {node: np.mean(crowding[node]) for node in crowding if crowding[node]}
nodes = G.nodes(data=True)

# people density at public nodes (waiting areas, corridors and stairs)
keywords = ['WAITING', 'CORRIDOR', 'STAIR']

filtered_node_ids = [
    node_id for node_id, attributes in nodes 
    if any(keyword in attributes.get('name', '') for keyword in keywords)
]

filtered_crowding = {
    node_id: Average_crowding_at_each_node[node_id] 
    for node_id in filtered_node_ids 
    if node_id in Average_crowding_at_each_node
}

# average people density at public areas (waiting areas, corridors and stairs)
average_crowding = sum(filtered_crowding.values()) / len(filtered_crowding)

#print("Average waiting times at each node:", {node: np.mean(waiting_times[node]) for node in waiting_times if waiting_times[node]})
print("Average patient walking distance:", average_walking_distance)
print("Average patient waiting time:", average_patient_waiting_time)
print("Average people density at public areas:", average_crowding)
print("Average extra walking distance:", extra_distance)


Average patient walking distance: 168.17482525482637
Average patient waiting time: 102.64097429122174
Average people density at public areas: 0.0759929391574581
Average extra walking distance: 23847.378656895384


## Reletavizing simulation results

### Relativizing average patient walking distance

### Relative Patient Walking Distance = (Average Patient Walking Distance/ graph Total Distance)×100

In [518]:
# Calculate the distance between each pair of connected nodes in the graph
def calculate_total_path_distance(graph):
    total_distance = 0
    for u, v in graph.edges:
        pos_u = np.array(graph.nodes[u]['coords'])
        pos_v = np.array(graph.nodes[v]['coords'])
        # Calculate Euclidean distance between nodes u and v
        distance = np.linalg.norm(pos_u - pos_v)
        total_distance += distance
    return total_distance

# Total path distance for the entire hospital layout
total_path_distance = calculate_total_path_distance(G)

# Calculate relative walking distance as a percentage of total path distance
relative_walking_distance = (average_walking_distance / total_path_distance) * 100

print(f"Total Path Distance (hospital size): {total_path_distance:.2f}")
print(f"Relative Walking Distance (% of hospital size): {relative_walking_distance:.2f}%")


Total Path Distance (hospital size): 1604.18
Relative Walking Distance (% of hospital size): 10.48%


### Relativizing average patient extra walking distance

### Relative Patient Extra Walking Distance = (Average Patient Extra Walking Distance/ graph Total Distance)×100

In [519]:
relative_extra_walking_distance = (extra_distance / total_path_distance) * 100

print(f"Relative Patient Extra Walking Distance (% of hospital size): {relative_extra_walking_distance:.2f}%")


Relative Patient Extra Walking Distance (% of hospital size): 1486.58%


## Interpretating simulation results